# Developing a Machine Learning Model to Localize Sensor Nodes in a Wireless Sensor Network

ToDo: 
1. Hyperparameter tuning
2. Test the models on different buildings and floors
3. Have working models by Thursday the 14th

In [1]:
import pandas as pd


C:\Users\gps18.REMAINHEALTHY\AppData\Local\Temp\ipykernel_15888\2035458946.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


(3.4640801314923673, 121)